In [1]:
import pandas as pd
import numpy as np


In [2]:
# читаем данные из csv, который находится в zip-файле на github
url = 'https://github.com/nikolai-karpov/sf_data_science/blob/943d3d51c5892ae950f606aa1ebd6249246f8dfb/Jupyter%20Notebook/data/diabetes_data.zip?raw=true'
df = pd.read_csv(url, encoding='ISO-8859-1', compression='zip')

In [3]:
# смотрим первые 5 строк таблицы
display(df.head())
# смотрим последние 5 строк таблицы
display(df.tail())

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome,Gender
0,6,98,58,33,190,34.0,0.430,43,0,Female
1,2,112,75,32,0,35.7,0.148,21,0,Female
2,2,108,64,0,0,30.8,0.158,21,0,Female
3,8,107,80,0,0,24.6,0.856,34,0,Female
4,7,136,90,0,0,29.9,0.210,50,0,Female


,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome,Gender
773,6,103,72,32,190,37.7,0.324,55,0,Female
774,1,71,48,18,76,20.4,0.323,22,0,Female
775,0,117,0,0,0,33.8,0.932,44,0,Female
776,4,154,72,29,126,31.3,0.338,37,0,Female
777,5,147,78,0,0,33.7,0.218,65,0,Female


Признаки данных
---
*   Pregnancies — количество беременностей.
*   Glucose — концентрация глюкозы в плазме через два часа при пероральном тесте на толерантность к глюкозе
*   BloodPressure — диастолическое артериальное давление (мм рт. ст.).
*   SkinThickness — толщина кожной складки трицепса (мм).
*   Insulin — двухчасовой сывороточный инсулин (ме Ед/мл).
*   BMI — индекс массы тела
*   DiabetesPedigreeFunction — функция родословной диабета (чем она выше, тем выше шанс наследственной заболеваемости).
*   Age — возраст.
*   Outcome — наличие диабета (0 — нет, 1 — да).


Задание 8.1 Найти все повторяющиеся строки в данных и удалить их

In [4]:
duplicates = df[df.duplicated()]
print('Число дубликтов: {}'.format(duplicates.shape[0]))
df = df.drop_duplicates()
print('Результирующее число записей: {}'.format(df.shape[0]))

Число дубликтов: 10
Результирующее число записей: 768


Задание 8.2 найдите все неинформативные признаки в данных и избавьтесь от них
---
В качестве порога информативности возьмите 0.99: 
удалите все признаки, для которых 99 % значений повторяются или 99 % записей уникальны.

In [5]:
# список неинформативных признаков
low_information_cols = [] 

# цикл по всем столбцам
for col in df.columns:
    # Доля от общих данных, которую занимает каждое уникальное значение в признаке 
    top_freq = df[col].value_counts(normalize=True).max()      # наибольшая относительная частота повторений
    # Отношение числа уникальных значений в столбце к размеру всего столбца
    nunique_ratio = df[col].nunique() / df[col].count() # Делим число уникальных значений на кол-во (1 все уникально, или есть дубликаты)
    # сравниваем наибольшую частоту с порогом
    if top_freq > 0.99:
        low_information_cols.append(col)                              # Если кол-во дубликатов выше указанного порога (0.99) заносим название столбца в список
        print(f'{col}: {round(top_freq*100, 2)}% одинаковых значений')
    # сравниваем долю уникальных значений с порогом
    if nunique_ratio > 0.99:
        low_information_cols.append(col)                              # Если уникальность выше указанного порога (0.99) заносим название столбца в список
        print(f'{col}: {round(nunique_ratio*100, 2)}% уникальных значений')

Gender: 100.0% одинаковых значений


In [6]:
cleaned_df = df.drop(low_information_cols, axis=1)
print(f'Результирующее число признаков: {cleaned_df.shape[1]}')

Результирующее число признаков: 9


Задание 8.3
---
Попробуйте найти пропуски в данных с помощью метода isnull().

Замените все записи, равные 0, в столбцах Glucose, BloodPressure, SkinThickness, Insulin и BMI на символ пропуска

In [7]:
def zero_replace(arg):
  return np.nan if arg == 0 else arg


cleaned_df['Glucose'] = cleaned_df['Glucose'].apply(zero_replace)
cleaned_df['BloodPressure'] = cleaned_df['BloodPressure'].apply(zero_replace)
cleaned_df['SkinThickness'] = cleaned_df['SkinThickness'].apply(zero_replace)
cleaned_df['Insulin'] = cleaned_df['Insulin'].apply(zero_replace)
cleaned_df['BMI'] = cleaned_df['BMI'].apply(zero_replace)

In [8]:
cols_null_percent = df.isnull().mean() * 100
cols_with_null = cols_null_percent[cols_null_percent>0].sort_values(ascending=False)
display(cols_with_null) # вывести на экран названия столбцов, где число пропусков больше 0.

Series([], dtype: float64)

In [9]:
#выводим результирующую долю пропусков
cleaned_df.isnull().mean().round(2).sort_values(ascending=False)

Insulin                     0.49
SkinThickness               0.30
BloodPressure               0.05
BMI                         0.01
Glucose                     0.01
Outcome                     0.00
Age                         0.00
DiabetesPedigreeFunction    0.00
Pregnancies                 0.00
dtype: float64

Задание 8.4
---
Удалите из данных признаки, где число пропусков составляет более 30 %. 

In [10]:
#задаем минимальный порог: вычисляем 70% от числа строк
thresh = df.shape[0]*0.7
#удаляем столбцы, в которых более 30% (100-70) пропусков
cleaned_df = cleaned_df.dropna(thresh=thresh, axis=1)
#отображаем результирующую долю пропусков
cleaned_df.isnull().mean()


Pregnancies                 0.000000
Glucose                     0.006510
BloodPressure               0.045573
SkinThickness               0.295573
BMI                         0.014323
DiabetesPedigreeFunction    0.000000
Age                         0.000000
Outcome                     0.000000
dtype: float64

Задание 8.5
---
Удалите из данных только те строки, в которых содержится более двух пропусков одновременно.

In [11]:
cleaned_df

,Pregnancies,Glucose,BloodPressure,SkinThickness,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,98.0,58.0,33.0,34.0,0.430,43,0
1,2,112.0,75.0,32.0,35.7,0.148,21,0
2,2,108.0,64.0,NaN,30.8,0.158,21,0
3,8,107.0,80.0,NaN,24.6,0.856,34,0
4,7,136.0,90.0,NaN,29.9,0.210,50,0
...,...,...,...,...,...,...,...,...
763,5,139.0,64.0,35.0,28.6,0.411,26,0
764,1,96.0,122.0,NaN,22.4,0.207,27,0
765,10,101.0,86.0,37.0,45.6,1.136,38,1
766,0,141.0,NaN,NaN,42.4,0.205,29,1


In [12]:
m = cleaned_df.shape[1]
cleaned_df = cleaned_df.dropna(thresh=m-2, axis=0)
print(cleaned_df.shape[0])

761


Задание 8.6
---
В оставшихся записях замените пропуски на медиану. 

In [14]:
"""Решение из подсказки
null_data = cleaned_df.isnull().sum()
cols = null_data[null_data>0].index
for col in cols:
    cleaned_df[col] = cleaned_df[col].fillna(cleaned_df[col].median())
print(round(cleaned_df['SkinThickness'].mean()))"""

29


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


In [15]:
#создаем словарь имя столбца: число(признак) на который надо заменить пропуски
values = {
    'Pregnancies': cleaned_df['Pregnancies'].median(),
    'Glucose': cleaned_df['Glucose'].median(),
    'BloodPressure': cleaned_df['BloodPressure'].median(),
    'SkinThickness': cleaned_df['SkinThickness'].median(),
    'BMI': cleaned_df['BMI'].median(),
    'DiabetesPedigreeFunction': cleaned_df['DiabetesPedigreeFunction'].median(),
    'Age': cleaned_df['Age'].median(),
    'Outcome': cleaned_df['Outcome'].median()
}
#заполняем пропуски в соответствии с заявленным словарем
cleaned_df = cleaned_df.fillna(values)

In [16]:
cleaned_df['SkinThickness'].median()

29.0

Задание 8.7
---
Сколько выбросов найдёт классический метод межквартильного размаха в признаке SkinThickness?

In [24]:
def outliers_iqr_mod(data, feature, left=1.5, right=1.5, log_scale=False):
    if log_scale:
        x = np.log(data[feature])
    else:
        x= data[feature]
    quartile_1, quartile_3 = x.quantile(0.25), x.quantile(0.75),
    iqr = quartile_3 - quartile_1
    lower_bound = quartile_1 - (iqr * left)
    upper_bound = quartile_3 + (iqr * right)
    outliers = data[(x<lower_bound) | (x > upper_bound)]
    cleaned = data[(x>lower_bound) & (x < upper_bound)]
    return outliers, cleaned

    
outliers, _ = outliers_iqr_mod(cleaned_df, 'SkinThickness')
print(outliers.shape[0])

87


Задание 8.8
---
Сколько выбросов найдёт классический метод z-отклонения в признаке SkinThickness?

In [27]:
def outliers_z_score(data, feature, log_scale=False):
    if log_scale:
        x = np.log(data[feature]+1)
    else:
        x = data[feature]
    mu = x.mean()
    sigma = x.std()
    lower_bound = mu - 3 * sigma
    upper_bound = mu + 3 * sigma
    outliers = data[(x < lower_bound) | (x > upper_bound)]
    cleaned = data[(x > lower_bound) & (x < upper_bound)]
    return outliers, cleaned


outliers, cleaned = outliers_z_score(cleaned_df, 'SkinThickness')
print(f'Число выбросов по методу z-отклонения: {outliers.shape[0]}')
print(f'Результирующее число записей: {cleaned.shape[0]}')

Число выбросов по методу z-отклонения: 4
Результирующее число записей: 757


Задание 8.9
---
Найдите число выбросов в признаке DiabetesPedigreeFunction с помощью классического метода межквартильного размаха.

In [28]:
def outliers_iqr_mod(data, feature, left=1.5, right=1.5, log_scale=False):
    if log_scale:
        x = np.log(data[feature])
    else:
        x= data[feature]
    quartile_1, quartile_3 = x.quantile(0.25), x.quantile(0.75),
    iqr = quartile_3 - quartile_1
    lower_bound = quartile_1 - (iqr * left)
    upper_bound = quartile_3 + (iqr * right)
    outliers = data[(x<lower_bound) | (x > upper_bound)]
    cleaned = data[(x>lower_bound) & (x < upper_bound)]
    return outliers, cleaned


outliers, cleaned = outliers_iqr_mod(cleaned_df, 'DiabetesPedigreeFunction')
print(f'Число выбросов по методу z-отклонения: {outliers.shape[0]}')
print(f'Результирующее число записей: {cleaned.shape[0]}')

Число выбросов по методу z-отклонения: 29
Результирующее число записей: 732


In [29]:
def outliers_iqr_mod(data, feature, left=1.5, right=1.5, log_scale=False):
    if log_scale:
        x = np.log(data[feature])
    else:
        x= data[feature]
    quartile_1, quartile_3 = x.quantile(0.25), x.quantile(0.75),
    iqr = quartile_3 - quartile_1
    lower_bound = quartile_1 - (iqr * left)
    upper_bound = quartile_3 + (iqr * right)
    outliers = data[(x<lower_bound) | (x > upper_bound)]
    cleaned = data[(x>lower_bound) & (x < upper_bound)]
    return outliers, cleaned


outliers, cleaned = outliers_iqr_mod(cleaned_df, 'DiabetesPedigreeFunction', log_scale=True)
print(f'Число выбросов по методу z-отклонения: {outliers.shape[0]}')
print(f'Результирующее число записей: {cleaned.shape[0]}')

Число выбросов по методу z-отклонения: 0
Результирующее число записей: 761
